In [2]:
import pandas as pd
import MySQLdb
import os
import time

In [3]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua","iron_ore_origional")
db.set_character_set("utf8")

In [3]:
data = pd.read_csv('../大商所/铁矿石/20171030.csv',engine='python',index_col=0,encoding='gbk')

In [4]:
data

,总成交量,增减,总持买单量,增减.1,总持卖单量,增减.2,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
会员类别,,,,,,,,,,,
期货公司会员,"2,687,776","-970,450","1,341,854","-48,413","1,341,854","-48,413",NaN,NaN,NaN,NaN,NaN
非期货公司会员,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
名次,会员简称,成交量,增减,名次,会员简称,持买单量,增减,名次,会员简称,持卖单量,增减
1,国投安信,"257,663","-7,980",1,中信期货,"83,970",65,1,中信期货,"115,194","-1,238"
2,海通期货,"158,652","-50,081",2,银河期货,"68,884","-9,758",2,永安期货,"95,844",271
3,东证期货,"102,246","-22,631",3,永安期货,"64,019","1,906",3,银河期货,"63,610","-4,856"
4,申银万国,"98,249","-9,536",4,海通期货,"54,871","-4,365",4,国投安信,"61,239","1,512"
5,银河期货,"98,048","-26,241",5,国投安信,"53,015","-4,965",5,方正中期,"51,449","1,280"
6,光大期货,"95,248","-14,236",6,国泰君安,"48,722","-3,112",6,申银万国,"48,268","-1,265"


# 原油

In [3]:
DATA_DIR = "../CL Daily Data WTI原油期货-所有资料"
for filename in os.listdir(DATA_DIR):
      
    cursor = db.cursor()
    sql = "CREATE DATABASE IF NOT EXISTS Oil_original_data"
    cursor.execute(sql)
    
    contract = filename.split(".xlsx")[0]
    
    sql = "CREATE TABLE IF NOT EXISTS Oil_original_data." + contract + \
          " (Date DATE,\
            Open FLOAT,\
            High FLOAT,\
            Low FLOAT,\
            Close FLOAT,\
            Volume FLOAT,\
            OI FLOAT)"
    cursor.execute(sql)
    
    
    data = pd.read_excel(DATA_DIR+"/"+filename,header=None)
    data = data.fillna("NULL")
    
    
    for i in range(8,len(data)):
        cursor = db.cursor()
        sql = "insert into Oil_original_data." + contract + " values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                + str(data.iloc[i][1]) + ","\
                + str(data.iloc[i][2]) + ","\
                + str(data.iloc[i][3]) + ","\
                + str(data.iloc[i][4]) + ","\
                + str(data.iloc[i][5]) + ","\
                + str(data.iloc[i][6]) + ")"
        index = i
        cursor.execute(sql)
    cursor.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: Warning: (1007, "Can't create database 'oil_original_data'; database exists")
  


# 鐵礦石

In [4]:
DATA_DIR = "../all IOE"
for filename in os.listdir(DATA_DIR):
      
    cursor = db.cursor()
    sql = "CREATE DATABASE IF NOT EXISTS Iron_original_data"
    cursor.execute(sql)
    
    contract = filename.split(".xlsx")[0]
    
    sql = "CREATE TABLE IF NOT EXISTS Iron_original_data." + contract + \
          " (Date DATE,\
            Open FLOAT,\
            High FLOAT,\
            Low FLOAT,\
            Close FLOAT,\
            Volume FLOAT,\
            OI FLOAT)"
    cursor.execute(sql)
    
    
    data = pd.read_excel(DATA_DIR+"/"+filename,header=None)
    data = data.fillna("NULL")
    
    
    for i in range(1,len(data)):
        cursor = db.cursor()
        sql = "insert into Iron_original_data." + contract + " values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                + str(data.iloc[i][1]) + ","\
                + str(data.iloc[i][2]) + ","\
                + str(data.iloc[i][3]) + ","\
                + str(data.iloc[i][4]) + ","\
                + str(data.iloc[i][5]) + ","\
                + str(data.iloc[i][6]) + ")"
        index = i
        cursor.execute(sql)
    cursor.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: Warning: (1007, "Can't create database 'iron_original_data'; database exists")
  


In [3]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")
data = pd.read_excel('../all IOE/IOEZ16.xlsx',header=None)
data = data.fillna("NULL")
cursor = db.cursor()
  
sql = "CREATE TABLE IF NOT EXISTS Oil_original_data.IOEZ16 "\
        " (Date DATE,\
        Open FLOAT,\
        High FLOAT,\
        Low FLOAT,\
        Close FLOAT,\
        Volume FLOAT,\
        OI FLOAT)"
cursor.execute(sql)
    
for i in range(1,len(data)):
    cursor = db.cursor()
    sql = "insert into Oil_original_data.ioez16 values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                + str(data.iloc[i][1]) + ","\
                + str(data.iloc[i][2]) + ","\
                + str(data.iloc[i][3]) + ","\
                + str(data.iloc[i][4]) + ","\
                + str(data.iloc[i][5]) + ","\
                + str(data.iloc[i][6]) + ")"
        
    cursor.execute(sql)
db.commit()